In [1]:
# Check device number
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Jun 24 01:25 /dev/video0


In [2]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera

# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
# camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

camera created


In [3]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'uangEfficientNet-B0'

CATEGORIES = ['Rp100k', 'Rp50k', 'Rp20k', 'Rp10k', 'Rp5k', 'Rp2k', 'Rp1k']  # Ubah titik (.) ke underscore (_) atau hapus

DATASETS = ['F']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset('../data/classification/' + TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))


uangEfficientNet-B0 task with ['Rp100k', 'Rp50k', 'Rp20k', 'Rp10k', 'Rp5k', 'Rp2k', 'Rp1k'] categories defined


In [4]:
# Set up the data directory location if not there already
DATA_DIR = '/nvdli-nano/data/classification/'
!mkdir -p {DATA_DIR}

In [5]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

data_collection_widget created


In [6]:
import torch
import torchvision
import ipywidgets
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# EfficientNet-B0
model = torchvision.models.efficientnet_b0(pretrained=True)

# Mengubah Fully Connected Layer terakhir agar sesuai dengan jumlah kategori
model.classifier[1] = nn.Linear(1280, len(dataset.categories))  # 1280 adalah fitur output dari EfficientNet-B0

model = model.to(device)

# Widget untuk menyimpan dan memuat model
model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='/nvdli-nano/data/classification/my_model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
    model.eval()
    print("Model EfficientNet-B0 berhasil dimuat!")
model_load_button.on_click(load_model)

def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
    print("Model EfficientNet-B0 berhasil disimpan!")
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])

# display(model_widget)
print("EfficientNet-B0 model configured and model_widget created")


EfficientNet-B0 model configured and model_widget created


In [7]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F
import ipywidgets
from IPython.display import display
import subprocess

# --- KONFIGURASI ---
LOG_FILE = 'debug_log.txt'
stop_log_thread = threading.Event()
# ------------------

# --- FUNGSI TEXT TO SPEECH ---

def speak_rupiah_subprocess(text, log_file_handle):
    try:
        log_file_handle.write(f"--> TTS: Mencoba mengucapkan: '{text}'\n")
        command = ['espeak-ng', '-a', '200', '-s', '140', text]
        
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

        log_file_handle.write(f"--> TTS: Perintah 'espeak-ng' berhasil dieksekusi.\n")
        
    except FileNotFoundError:
        log_file_handle.write("--> TTS ERROR: Perintah 'espeak-ng' tidak ditemukan.\n")
    except subprocess.CalledProcessError as e:
        log_file_handle.write(f"--> TTS ERROR: Perintah 'espeak-ng' gagal.\n")
        log_file_handle.write(f"    Pesan Error dari espeak-ng: {e.stderr}\n")
    except Exception as e:
        log_file_handle.write(f"--> TTS ERROR: Terjadi error tak terduga: {e}\n")

# --- Konfigurasi Lainnya ---
currency_speech_map = { "Rp100k": "One Hundred Thousand ", "Rp50k": "Fifty Thousand ", "Rp20k": "Twenty Thousand ", "Rp10k": "Ten Thousand ", "Rp5k": "Five Thousand ", "Rp2k": "Two Thousand ", "Rp1k": "One Thousand " }
CONFIDENCE_THRESHOLD = 0.90
last_spoken_currency = None

low_confidence_start_time = None  # Untuk melacak kapan mulai tidak melihat uang
RESET_SECONDS = 1                 # Atur waktu tunggu (dalam detik)

# --- Widget UI ---
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction', disabled=True)
score_widgets = []
for category in dataset.categories:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, description=category, orientation='vertical', disabled=True)
    score_widgets.append(score_widget)
log_display_widget = ipywidgets.Textarea(value='Log debug akan muncul di sini...\n', description='Debug Log:', layout={'width': '99%', 'height': '200px'}, disabled=True)


def live(state_widget, model_obj, camera_obj, prediction_disp_widget, score_disp_widgets_list):
    global dataset, last_spoken_currency, device, low_confidence_start_time
    
    model_obj.eval()
    while state_widget.value == 'live':
        try:
            with open(LOG_FILE, 'a') as log_f:
                image = camera_obj.value
                preprocessed_image = preprocess(image)
                with torch.no_grad():
                    output = model_obj(preprocessed_image)
                output_softmax = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
                
                category_index = output_softmax.argmax()
                predicted_label_str = dataset.categories[category_index]
                confidence_value = float(output_softmax[category_index])

                prediction_disp_widget.value = predicted_label_str
                for i, score_val in enumerate(list(output_softmax)):
                    score_disp_widgets_list[i].value = float(score_val)

                # --- LOGIKA BARU DENGAN TIMER RESET ---
                log_f.write("\n--- Frame Baru ---\n")
                log_f.write(f"Prediksi: {predicted_label_str}, Keyakinan: {confidence_value:.2f}, Terakhir Diucapkan: {last_spoken_currency}\n")

                is_confident = confidence_value >= CONFIDENCE_THRESHOLD
                is_in_map = predicted_label_str in currency_speech_map

                if is_confident and is_in_map:
                    # Jika kita melihat uang dengan yakin, reset timernya.
                    low_confidence_start_time = None
                    
                    is_new_currency = predicted_label_str != last_spoken_currency
                    if is_new_currency:
                        log_f.write("--> KEPUTUSAN: KONDISI TERPENUHI, MEMANGGIL FUNGSI SUARA.\n")
                        text_to_say = currency_speech_map[predicted_label_str]
                        speak_rupiah_subprocess(text_to_say, log_f)
                        last_spoken_currency = predicted_label_str
                    else:
                        log_f.write("--> KEPUTUSAN: TIDAK BICARA (Uang sama dengan yang terakhir diucapkan).\n")

                else: # Ini adalah frame dengan keyakinan rendah atau background
                    log_f.write("--> KEPUTUSAN: TIDAK BICARA (Keyakinan rendah atau bukan uang).\n")
                    if last_spoken_currency is not None: # Hanya jalankan timer jika kita perlu me-reset sesuatu
                        if low_confidence_start_time is None:
                            # Ini frame pertama kita tidak melihat uang, mulai timer.
                            log_f.write("    -> Timer reset dimulai...\n")
                            low_confidence_start_time = time.time()
                        else:
                            # Timer sudah berjalan, periksa apakah sudah waktunya reset.
                            elapsed_time = time.time() - low_confidence_start_time
                            log_f.write(f"    -> Waktu tunggu: {elapsed_time:.1f} dari {RESET_SECONDS} detik.\n")
                            if elapsed_time > RESET_SECONDS:
                                log_f.write("    -> WAKTU HABIS! Me-reset 'Uang Terakhir Diucapkan'.\n")
                                last_spoken_currency = None
                                low_confidence_start_time = None # Hentikan timer
        
        except Exception as e:
            with open(LOG_FILE, 'a') as log_f:
                log_f.write(f"\n!!! ERROR DALAM LOOP LIVE: {e} !!!\n")
        
        time.sleep(0.5) # Perlambat sedikit agar log lebih mudah dibaca

def update_log_display(stop_event):
    while not stop_event.is_set():
        try:
            with open(LOG_FILE, 'r') as f:
                log_display_widget.value = f.read()
        except FileNotFoundError:
            pass
        time.sleep(1)

def start_live(change):
    global last_spoken_currency, stop_log_thread
    if change['new'] == 'live':
        with open(LOG_FILE, 'w') as f:
            f.write("--- Sesi Live Baru Dimulai ---\n")
        
        last_spoken_currency = None
        live_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widgets))
        live_thread.daemon = True
        live_thread.start()
        
        stop_log_thread.clear()
        log_thread = threading.Thread(target=update_log_display, args=(stop_log_thread,))
        log_thread.daemon = True
        log_thread.start()
    else:
        stop_log_thread.set()

state_widget.observe(start_live, names='value')
live_execution_widget = ipywidgets.VBox([ipywidgets.HBox(score_widgets), prediction_widget, state_widget])
print("live_execution_widget (Final, dengan Log APPEND dan Perbaikan Subprocess) telah dikonfigurasi.")

live_execution_widget (Final, dengan Log APPEND dan Perbaikan Subprocess) telah dikonfigurasi.


In [1]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
accuracy_widget = ipywidgets.FloatText(description='accuracy')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, labels in iter(train_loader):
                # send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    ipywidgets.HBox([train_button, eval_button])
])

# display(train_eval_widget)
print("trainer configured and train_eval_widget created")

NameError: name 'torch' is not defined

In [9]:
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]),
    train_eval_widget,
    model_widget,
])

# Tampilkan widget utama, DAN di bawahnya tampilkan widget LOG DEBUG
display(all_widget, log_display_widget)

Textarea(value='Log debug akan muncul di sini...\n', description='Debug Log:', disabled=True, layout=Layout(he…